In [1]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==3.3.1

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==3.3.0

openjdk version "1.8.0_352"
OpenJDK Runtime Environment (build 1.8.0_352-8u352-ga-1~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.352-b08, mixed mode)
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 55.4 MB/s 
     |████████████████████████████████| 120 kB 4.3 MB/s 


In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,073 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:1

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-12 00:23:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2022-12-12 00:24:01 (1.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [7]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 
filtered_vine_df = vine_df.filter("total_votes>=20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [8]:
# Filter the new DataFrame and create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_votes_df = filtered_vine_df.withColumn("greater_50_percent", filtered_vine_df["helpful_votes"] / filtered_vine_df["total_votes"]).filter("greater_50_percent>=0.5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|greater_50_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|               0.8|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|0.6774193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|0.9642857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|               1.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|0.8729641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|0.8888888888888888|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|0.9583333333333334|
|R2JSWNTAO83X3V|          5|  

In [9]:
# Filter the DataFrame and create a new DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
Y_vine_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
Y_vine_review_df.show()

+---------+-----------+-------------+-----------+----+-----------------+------------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|greater_50_percent|
+---------+-----------+-------------+-----------+----+-----------------+------------------+
+---------+-----------+-------------+-----------+----+-----------------+------------------+



In [10]:
# Repeat the Step above, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
N_vine_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
N_vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|greater_50_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|               0.8|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|0.6774193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|0.9642857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|               1.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|0.8729641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|0.8888888888888888|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|0.9583333333333334|
|R2JSWNTAO83X3V|          5|  

In [11]:
# Determine the total number of reviews for the two types of review (paid vs unpaid).
Y_total_reviews_df = Y_vine_review_df.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Total_Reviews_Paid")
Y_total_reviews_df.show()
N_total_reviews_df = N_vine_review_df.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Total_Reviews_Unpaid")
N_total_reviews_df.show()

+------------------+
|Total_Reviews_Paid|
+------------------+
|                 0|
+------------------+

+--------------------+
|Total_Reviews_Unpaid|
+--------------------+
|              403807|
+--------------------+



In [12]:
# Determine the number of 5-star reviews for the two types of review (paid vs unpaid).
Y_5star_reviews_df = Y_vine_review_df.filter("star_rating==5").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "Total_5Star_Reviews_Paid")
Y_5star_reviews_df.show()
N_5star_reviews_df = N_vine_review_df.filter("star_rating==5").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "Total_5Star_Reviews_Unpaid")
N_5star_reviews_df.show()

+------------------------+
|Total_5Star_Reviews_Paid|
+------------------------+
|                       0|
+------------------------+

+--------------------------+
|Total_5Star_Reviews_Unpaid|
+--------------------------+
|                    242889|
+--------------------------+



In [13]:
# Determine the percentage of 5-star reviews for the two types of review (paid vs unpaid).
N_p5star_reviews = round((N_5star_reviews_df.collect()[0]["Total_5Star_Reviews_Unpaid"] / N_total_reviews_df.collect()[0]["Total_Reviews_Unpaid"] * 100), 2)
print("The percentage of 5-star unpaid book reviews is: %f" % N_p5star_reviews)

Y_p5star_reviews = round((Y_5star_reviews_df.collect()[0]["Total_5Star_Reviews_Paid"] / Y_total_reviews_df.collect()[0]["Total_Reviews_Paid"] * 100),2)
print("The percentage of 5-star paid book reviews is: %f" % Y_p5star_reviews)

The percentage of 5-star unpaid book reviews is: 60.150000


ZeroDivisionError: ignored